In [2]:
import re
import pandas as pd
import numpy as np
import ast
import operator


def union_num_index_syn_rule_out(dic_syn,list_index_rule_out):
    _dic_syn_=dic_syn.copy()
    for row,dic in enumerate(_dic_syn_):
        synset=set()
        for idx,syn in dic.items():
            if idx not in list_index_rule_out[row]:
                synset=synset.union(set(syn))
        _dic_syn_[row]=synset
    return(_dic_syn_) 


def union_num_index_syn_keep_in(dic_syn,list_index_rule_out):
    _dic_syn_=dic_syn.copy()
    for row,dic in enumerate(_dic_syn_):
        synset=set()
        for idx,syn in dic.items():
            if idx in list_index_rule_out[row]:
                synset=synset.union(set(syn))
        _dic_syn_[row]=synset
    return(_dic_syn_) 



def update_table_with_weights_and_remove_sets(words_table,columns,remove_sets):
    table=words_table.copy()
    for i in range(len(table)):
        for col in columns:
            table[col[1]][i]=set(table[col[0]][i])
            table[col[1]][i]=table[col[1]][i].difference(table[remove_sets[0]][i])
            table[col[1]][i]=table[col[1]][i].difference(table[remove_sets[1]][i])
            table[col[1]][i]=list(table[col[1]][i])
            table[col[1]][i]=table[col[1]][i]*col[2]
    return(table)
        

def union_syn_cols(dataset,features,union_feature):
    _dataset_=dataset.copy()
    _dataset_[union_feature]=""
    for row in range(len(_dataset_[union_feature])):
        synlist=[]
        for feature in features:
            synlist+=_dataset_[feature][row]
        _dataset_[union_feature][row]=synlist
    return(_dataset_[union_feature])

def synonym_voter(dataset,union_feature):
    _dataset_=dataset.copy()
    _dataset_["temp"]=""
    for row in range(0,len(_dataset_[union_feature])):
        ranked_dic={}
        for syn in _dataset_[union_feature][row]:
            if syn in ranked_dic:
                ranked_dic[syn]+=1
            else:
                ranked_dic[syn]=1
        ranked_dic = sorted(ranked_dic.items(), key=operator.itemgetter(1),reverse=True)
        _dataset_["temp"][row]=ranked_dic
    return(_dataset_["temp"])
        

In [3]:
words_table=pd.read_excel(
    "./excels/2_Phase3_Word_Sense_Voting_CJ-15-07-19.xlsx")

columns=['JDM_Sense',
         'JDM',
         'WordNet',
         'Cnrtl',
         'Synonymo',
         'Cisco',
         'DicSyn_Sense']



words_table["JDM sense to drop"]=words_table["JDM sense to drop"].astype(str).str.replace(".",",").str.replace("nan","")
words_table["JDM sense to drop"]=words_table["JDM sense to drop"].apply(lambda s : re.sub("(^.*$)","[\\1]",s))
words_table["JDM sense to drop"]=words_table["JDM sense to drop"].apply(lambda s: ast.literal_eval(s))


words_table["DicSyn sense to drop"]=words_table["DicSyn sense to drop"].astype(str).str.replace(".",",").str.replace("nan","")
words_table["DicSyn sense to drop"]=words_table["DicSyn sense to drop"].apply(lambda s : re.sub("(^.*$)","[\\1]",s))
words_table["DicSyn sense to drop"]=words_table["DicSyn sense to drop"].apply(lambda s: ast.literal_eval(s))


words_table=words_table.fillna("[]")
words_table["DicSyn_Sense"]=words_table["DicSyn_Sense"].apply(lambda s: re.sub("\[\]", '{}', s))
words_table["JDM_Sense"]=words_table["JDM_Sense"].apply(lambda s: re.sub("\[\]", '{}', s))

for col in columns:
    words_table[col]=words_table[col].apply(lambda s: re.sub(",[\s]*nan", ' ', s))
    words_table[col]=words_table[col].apply(lambda s: re.sub("[^,\s]*nan[\s]+,", '[', s))
    words_table[col]=words_table[col].apply(lambda s: ast.literal_eval(s))
    words_table[col+"_"]=""
    

words_table["DicSyn_Sense_"]=union_num_index_syn_rule_out(words_table["DicSyn_Sense"],words_table["DicSyn sense to drop"])  
words_table["JDM_Sense_"]=union_num_index_syn_rule_out(words_table["JDM_Sense"],words_table["JDM sense to drop"])  

words_table["DicSyn_Sense_"]=words_table["DicSyn_Sense_"].apply(lambda s: list(s)) 
words_table["JDM_Sense_"]=words_table["JDM_Sense_"].apply(lambda s: list(s)) 


words_table["DicSyn_remove"]=union_num_index_syn_keep_in(words_table["DicSyn_Sense"],words_table["DicSyn sense to drop"])  
words_table["JDM_remove"]=union_num_index_syn_keep_in(words_table["JDM_Sense"],words_table["JDM sense to drop"])  

remove_sets=["DicSyn_remove","JDM_remove"]

columns=[['JDM_Sense_'   ,'JDM_Sense_'   ,3],
         ['JDM'          ,'JDM_'         ,2],
         ['WordNet'      ,'WordNet_'     ,2],
         ['Cnrtl'        ,'Cnrtl_'       ,1],
         ['Synonymo'     ,'Synonymo_'    ,2],
         ['Cisco'        ,'Cisco_'       ,2],
         ['DicSyn_Sense_','DicSyn_Sense_',3]
        ]

words_table=update_table_with_weights_and_remove_sets(words_table,columns,remove_sets)

words_table["Union_Sense"]=union_syn_cols(words_table,
                                      ['JDM_Sense_','JDM_','WordNet_','Cnrtl_','Synonymo_','Cisco_','DicSyn_Sense_']
                                      ,"Union_Sense")

words_table["Synonyms_Voting"]=synonym_voter(words_table,"Union_Sense")

words_table_to_write=words_table[['Word','Word_Corrected','JDM_Sense_','JDM sense to drop','JDM_','WordNet_','Cnrtl_','Synonymo_','Cisco_','DicSyn_Sense_','DicSyn sense to drop','Synonyms_Voting']]


words_table_to_write["Synonyms_Voting_Top5"]=""
words_table_to_write["Synonyms_Voting_Top10"]=""
words_table_to_write["Synonyms_Voting_Top15"]=""
words_table_to_write["Synonyms_Voting_Top20"]=""
words_table_to_write["Synonyms_Voting_Rest"]=""

for row in range(len(words_table_to_write["Synonyms_Voting"])):
    words_table_to_write["Synonyms_Voting_Top5"][row]=[x for x,y in words_table_to_write["Synonyms_Voting"][row][0:5]]
    words_table_to_write["Synonyms_Voting_Top10"][row]=[x for x,y in words_table_to_write["Synonyms_Voting"][row][0:10]]
    words_table_to_write["Synonyms_Voting_Top15"][row]=[x for x,y in words_table_to_write["Synonyms_Voting"][row][0:15]]
    words_table_to_write["Synonyms_Voting_Top20"][row]=[x for x,y in words_table_to_write["Synonyms_Voting"][row][0:20]]
    words_table_to_write["Synonyms_Voting_Rest"][row]=[x for x,y in words_table_to_write["Synonyms_Voting"][row][20:]]


# words_table_to_write.to_excel(
#     "./excels/3_Phase3_Word_Sense_Voting_New_Format.xlsx")

# words_table_to_write  

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [38]:
words=pd.read_excel(
    "./excels/1_Phase3_TranslationTable.xlsx")

words_table_to_write["List_of_Occurances"]=""

for i in range(len(words_table_to_write["Word"])):
    words_list=[]
    for row,value in enumerate(words["French Expression"]):
        if words_table_to_write["Word"][i] in value:
            words_list.append(value)    
    words_table_to_write["List_of_Occurances"][i]=words_list
            
words_table_to_write.to_excel(
    "./excels/3_Phase3_Word_Sense_Voting_with_Occurances.xlsx")    
    